# LG-CoTrain: All Disasters Experiment

This notebook runs the **LG-CoTrain** co-training pipeline across **all 10 disaster
events**, each with **4 budgets x 3 seed sets = 12 experiments**, for a total of
**120 experiments**.

### Resume Support (two levels)

1. **Event level**: Before starting an event, we check if all 12 `metrics.json`
   files already exist. If so, the entire event is skipped.
2. **Experiment level**: Within each event, `run_all_experiments` skips individual
   `(budget, seed_set)` combinations that already have `metrics.json`.

This means you can safely **restart the notebook after a crash** and it will
resume from where it left off.

### Events

| # | Event |
|---|-------|
| 1 | california_wildfires_2018 |
| 2 | canada_wildfires_2016 |
| 3 | cyclone_idai_2019 |
| 4 | hurricane_dorian_2019 |
| 5 | hurricane_florence_2018 |
| 6 | hurricane_harvey_2017 |
| 7 | hurricane_irma_2017 |
| 8 | hurricane_maria_2017 |
| 9 | kaikoura_earthquake_2016 |
| 10 | kerala_floods_2018 |

In [ ]:
import json
import statistics
import sys
import time
from pathlib import Path

def _find_repo_root(marker: str = "lg_cotrain") -> Path:
    for candidate in [Path().resolve()] + list(Path().resolve().parents):
        if (candidate / marker).is_dir():
            return candidate
    raise RuntimeError(
        f"Cannot find repo root: no ancestor directory contains '{marker}/'. "
        "Run the notebook from inside the repository."
    )

repo_root = _find_repo_root()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import matplotlib.pyplot as plt
import numpy as np

from lg_cotrain.run_all import BUDGETS, SEED_SETS, run_all_experiments, format_summary_table

DATA_ROOT = str(repo_root / "data")
RESULTS_ROOT = str(repo_root / "results")

print(f"Repo root: {repo_root}")
print(f"Data root: {DATA_ROOT}")
print(f"Results root: {RESULTS_ROOT}")
print(f"Budgets: {BUDGETS}")
print(f"Seed sets: {SEED_SETS}")
print(f"Experiments per event: {len(BUDGETS) * len(SEED_SETS)}")

In [ ]:
def is_event_complete(event, results_root):
    """Check if all 12 metrics.json files exist for an event."""
    for budget in BUDGETS:
        for seed_set in SEED_SETS:
            path = Path(results_root) / event / f"{budget}_set{seed_set}" / "metrics.json"
            if not path.exists():
                return False
    return True

# Discover events from data directory
data_dir = Path(DATA_ROOT) / "original"
all_events = sorted(p.name for p in data_dir.iterdir() if p.is_dir())

completed_events = [e for e in all_events if is_event_complete(e, RESULTS_ROOT)]
pending_events = [e for e in all_events if e not in completed_events]

print(f"Found {len(all_events)} events total")
print(f"  Completed: {len(completed_events)} ({len(completed_events) * 12} experiments)")
print(f"  Pending:   {len(pending_events)} (up to {len(pending_events) * 12} experiments)")

if completed_events:
    print(f"\nCompleted events (will be skipped):")
    for e in completed_events:
        print(f"  - {e}")

if pending_events:
    print(f"\nPending events (will be run):")
    for e in pending_events:
        print(f"  - {e}")

## Running Experiments

For each pending event, we call `run_all_experiments` which runs all 12
`(budget, seed_set)` combinations. Individual experiments that already have
`metrics.json` are automatically skipped (useful if the notebook crashed
mid-event).

After each event completes, its summary table is printed immediately so
results are visible even if the notebook is interrupted later.

In [ ]:
all_event_results = {}
total_events = len(pending_events)
overall_start = time.time()

# Run pending events
for i, event in enumerate(pending_events, 1):
    print(f"\n{'=' * 60}")
    print(f"Event {i}/{total_events}: {event}")
    print(f"{'=' * 60}")

    results = run_all_experiments(
        event,
        data_root=DATA_ROOT,
        results_root=RESULTS_ROOT,
    )
    all_event_results[event] = results

    print()
    print(format_summary_table(results, event))

# Load results for already-completed events
for event in completed_events:
    results = []
    for budget in BUDGETS:
        for seed_set in SEED_SETS:
            path = Path(RESULTS_ROOT) / event / f"{budget}_set{seed_set}" / "metrics.json"
            with open(path) as f:
                results.append(json.load(f))
    all_event_results[event] = results

overall_elapsed = time.time() - overall_start
print(f"\n{'=' * 60}")
print(f"All events done in {overall_elapsed:.1f}s")
print(f"Total events with results: {len(all_event_results)}")

## Cross-Disaster Results

We now aggregate results across all 10 events to compare how the pipeline
performs on different disaster types and how performance scales with the
labeled data budget.

In [ ]:
# Build cross-disaster summary: event -> budget -> mean macro-F1
summary = {}
for event in sorted(all_event_results.keys()):
    results = all_event_results[event]
    by_budget = {b: [] for b in BUDGETS}
    for r in results:
        if r is not None:
            by_budget[r["budget"]].append(r)
    summary[event] = {}
    for b in BUDGETS:
        f1s = [r["test_macro_f1"] for r in by_budget[b]]
        errs = [r["test_error_rate"] for r in by_budget[b]]
        summary[event][b] = {
            "f1_mean": statistics.mean(f1s) if f1s else None,
            "f1_std": statistics.stdev(f1s) if len(f1s) >= 2 else None,
            "err_mean": statistics.mean(errs) if errs else None,
            "err_std": statistics.stdev(errs) if len(errs) >= 2 else None,
            "n_seeds": len(f1s),
        }

# Print grand summary table
header = f"{'Event':<35}"
for b in BUDGETS:
    header += f" | B={b:<11}"
print(header)
print("-" * len(header))

for event in sorted(summary.keys()):
    row = f"{event:<35}"
    for b in BUDGETS:
        s = summary[event][b]
        if s["f1_mean"] is not None and s["f1_std"] is not None:
            row += f" | {s['f1_mean']:.3f}+/-{s['f1_std']:.3f}"
        elif s["f1_mean"] is not None:
            row += f" | {s['f1_mean']:.3f}      "
        else:
            row += f" | {'N/A':<11}"
    print(row)

In [ ]:
# Line plot: Macro-F1 by budget, one line per event
fig, ax = plt.subplots(figsize=(10, 6))

for event in sorted(summary.keys()):
    means = [summary[event][b]["f1_mean"] or 0 for b in BUDGETS]
    stds = [summary[event][b]["f1_std"] or 0 for b in BUDGETS]
    ax.errorbar(BUDGETS, means, yerr=stds, marker="o", capsize=3, label=event)

ax.set_xlabel("Budget (labeled samples per class)")
ax.set_ylabel("Test Macro-F1 (mean +/- std across seeds)")
ax.set_title("LG-CoTrain Performance Across Disasters")
ax.set_xticks(BUDGETS)
ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap: events (rows) x budgets (columns), colored by mean macro-F1
events_sorted = sorted(summary.keys())
heatmap_data = np.zeros((len(events_sorted), len(BUDGETS)))

for i, event in enumerate(events_sorted):
    for j, b in enumerate(BUDGETS):
        val = summary[event][b]["f1_mean"]
        heatmap_data[i, j] = val if val is not None else 0

fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(heatmap_data, cmap="YlOrRd", aspect="auto")

ax.set_xticks(range(len(BUDGETS)))
ax.set_xticklabels([f"B={b}" for b in BUDGETS])
ax.set_yticks(range(len(events_sorted)))
ax.set_yticklabels(events_sorted, fontsize=9)
ax.set_title("Mean Test Macro-F1 by Event and Budget")

# Annotate cells with values
for i in range(len(events_sorted)):
    for j in range(len(BUDGETS)):
        val = heatmap_data[i, j]
        color = "white" if val > 0.6 else "black"
        ax.text(j, i, f"{val:.3f}", ha="center", va="center", color=color, fontsize=9)

fig.colorbar(im, ax=ax, label="Macro-F1")
plt.tight_layout()
plt.show()

## Summary

This notebook ran **120 experiments** (10 events x 4 budgets x 3 seed sets)
using the LG-CoTrain pipeline.

### Resume support
- Events with all 12 `metrics.json` files are automatically skipped
- Within partially-completed events, individual experiments with existing
  results are skipped
- Safe to restart after crashes

### CLI equivalent
```bash
# Run all 12 experiments for a single event
python -m lg_cotrain.run_all --event california_wildfires_2018
python -m lg_cotrain.run_all --event canada_wildfires_2016
# ... etc for all 10 events
```